# Jonathan Halverson
# Thursday, August 3, 2017
# Chapter 5 of Bruce and Bruce

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline
plt.style.use('halverson')

### Naive Bayes

Here we try to predict the nature of Wikipedia biographies using only three records per class to try the model.

In [3]:
import re
import requests
from bs4 import BeautifulSoup
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [4]:
def scrape_and_tokenize(person):
     # download and parse the biography
     base_url = 'https://en.wikipedia.org/wiki/'
     r = requests.get(base_url + person)
     soup = BeautifulSoup(r.content, 'lxml')

     # extract the text of each paragraph
     raw_text = ''
     for paragraph in soup.find_all('p'):
          raw_text += paragraph.get_text()
    
     # keep only alphabetical characters and split on whitespace
     letters_only = re.sub("[^a-zA-Z]", " ", raw_text)
     words = letters_only.lower().split()

     # count the words and filter based on count and stopwords, apply stemming
     count = Counter(words)
     porter = PorterStemmer()
     stops = stopwords.words("english")
     words = [porter.stem(word) for word in words if (word not in stops) and (count[word] > 1) and (len(word) > 1)]
     return words

In [5]:
einstein = scrape_and_tokenize('Albert_Einstein')
newton = scrape_and_tokenize('Isaac_Newton')
darwin = scrape_and_tokenize('Charles_Darwin')
spielberg = scrape_and_tokenize('Steven_Spielberg')
allen = scrape_and_tokenize('Woody_Allen')
cameron = scrape_and_tokenize('James_Cameron')
jordan = scrape_and_tokenize('Michael_Jordan')
brady = scrape_and_tokenize('Tom_Brady')
williams = scrape_and_tokenize('Serena_Williams')

In [6]:
einstein[:10]

[u'albert',
 u'einstein',
 u'german',
 u'march',
 u'april',
 u'german',
 u'born',
 u'theoret',
 u'physicist',
 u'einstein']

The idea of Naive Bayes is to calculate $P(Y|X_j)$ based on $P(X_j|Y)$. That is, knowin the probability of all the features being associated with a given class, given the features of a new record what is the most likely class? In this example, we have three records for each type of person. For each class we will compute the probably of each word being found. Then when a new biography is presented we will figure out which class it falls into based on the words in that record.

What is done below computes the probability based on the number times a word appears per record. The correct way of doing this is to work with the probability of a word appearing in a biography (i.e., one asks does the word 'data' appear in the biography or does it not).

In [7]:
from collections import Counter

c = Counter(einstein + newton + darwin + spielberg + allen + cameron + jordan + brady + williams)
c_scientist = Counter(einstein + newton + darwin)
c_filmmaker = Counter(spielberg + allen + cameron)
c_athlete = Counter(jordan + brady + williams)

In [8]:
k = 0.5
p_scientist = {}
for word, count in c_scientist.items():
     p_scientist[word] = (count + k) / float(c[word] + 2 * k)
p_filmmaker = {}
for word, count in c_filmmaker.items():
     p_filmmaker[word] = (count + k) / float(c[word] + 2 * k)
p_athlete = {}
for word, count in c_athlete.items():
     p_athlete[word] = (count + k) / float(c[word] + 2 * k)

In [9]:
p_scientist.items()[:10]

[(u'four', 0.13218390804597702),
 (u'captain', 0.8333333333333334),
 (u'whose', 0.6428571428571429),
 (u'deviat', 0.8333333333333334),
 (u'hermann', 0.875),
 (u'everi', 0.40476190476190477),
 (u'rise', 0.9166666666666666),
 (u'quantiz', 0.9),
 (u'govern', 0.6428571428571429),
 (u'disturb', 0.8333333333333334)]

When we consider new records we must keep in mind that there will be words that did not appear in the training set. These words must be ignored.

In [10]:
kubrick = scrape_and_tokenize('Stanley_Kubrick')
kubrick = [word for word in kubrick if word in c.keys()]

In [12]:
sum(np.log(p_scientist.values()))

-811.44216321963813

For a continuous feature, Gaussian Naive Bayes fit the feature to a Gaussian for each class and uses the pdf to generate the required conditional probabilities.

### Linear discriminant analysis

Find the weights that maximize class separability. The idea is to maximize the ratio of distance between the class centriods over the variance within each class, weighted by the covariance matrix. Bruce & Bruce treat it as a model whereas other sources treat it as a dimensionality reduction technique. There is also quadratic discriminant analysis.

LDA is used to find a linear combination of features that separates two or more classes. The resulting combination may be used as a linear classifier, or, more commonly, for dimensionality reduction before later classification.

The approach starts by assuming that the features are normally distributed. This gives QDA. The further assumption of the covariance of each class being is equal leads to LDA. The multivariate normal is specified by the mean of each feature and covariance matrix. Note that it does not assume that each feature within each class is normally distributed.

Notes on detecting multicollinearity: https://onlinecourses.science.psu.edu/stat501/node/347 See VIF or variance inflation factors.

### Strategies for imbalanced data

Whenever doing a classification problem one should always count the outcomes and see if they are balanced. Imagine the cause of fraud detection where there are 1 million zeroes and 100 ones. If all the data are used to fit the model, it is possible that some of the zeros cases will have very similar feature to the ones cases just by chance (since there are so many zero cases). This will make it unlikely for a model to be found which can distinguish between the classes.

If you have a vast number of records for each class, one could undersample by throwing away the records of the class with the most records.

The above approach throws away data which may not be ideal. Another approach is to weight the minority records so that the weights of the two are the same. So if there are 1000 zeroes and 25 ones, then weight the zeros with w=1 and the ones with weight w=40. The software must support this.

Note that it will always be interesting to compare the result of using an imbalanced class technique with using all the data. If you the result is the same it suggests that one of the class is quite difference from the other in terms of the features since chance does not cause the two to be indistiguishable.

Another approach is to increase the number of minority records by using bootstrapping. That is, draw a bootstrapped resample of the same size as the majority records. I don't like this as much as just weighting the minority since more computation is needed with this approach.

Lastly, there is data generation or methods like SMOTE. Here, new samples from the minority are created. Choose a minority record at random. Use KNN to find a similar minorty record. For each predictor choose a weight between 0 and 1 and create a linear combination with w and 1 - w as the weights between the two records.

### Cost-based classification

In the case of loan data, the idea is often to predict the probability of the loan being repaid versus defaulting. So the cost can be written as $C=P(y=0)R + P(y=1)D$, where R is the value gained on repayment and D is the value gained on default (D is less than zero). With estimates of R and D one can go one step further than just recall (sensitivity -- proportion of 1's correctly classified) and specificity to optimize the operating parameters of the model.